# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 15 2022 1:44PM,241983,10,CTF0011323,"Citieffe, Inc.",Released
1,Jul 15 2022 1:44PM,241983,10,CTF0011324,"Citieffe, Inc.",Released
2,Jul 15 2022 1:42PM,241982,10,8278850,"ClearSpec, LLC",Released
3,Jul 15 2022 1:34PM,241981,20,ALUR482796167,Alumier Labs Inc.,Released
4,Jul 15 2022 1:20PM,241979,21,25732-04,"NBTY Global, Inc.",Released
5,Jul 15 2022 1:04PM,241976,19,ACG-2102490620,ACG North America LLC,Released
6,Jul 15 2022 1:04PM,241976,19,ACG-2102490621,ACG North America LLC,Released
7,Jul 15 2022 11:21AM,241966,10,0085877494,ISDIN Corporation,Released
8,Jul 15 2022 11:21AM,241966,10,0085877511,ISDIN Corporation,Released
9,Jul 15 2022 11:21AM,241966,10,0085877514,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
12,241976,Released,2
13,241979,Released,1
14,241981,Released,1
15,241982,Released,1
16,241983,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
241976,NaN,2.0
241979,NaN,1.0
241981,NaN,1.0
241982,NaN,1.0
241983,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
241906,0.0,1.0
241912,1.0,0.0
241918,13.0,61.0
241919,0.0,1.0
241924,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
241906,0,1
241912,1,0
241918,13,61
241919,0,1
241924,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,241906,0,1
1,241912,1,0
2,241918,13,61
3,241919,0,1
4,241924,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,241906,,1
1,241912,1,
2,241918,13,61
3,241919,,1
4,241924,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 15 2022 1:44PM,241983,10,"Citieffe, Inc."
2,Jul 15 2022 1:42PM,241982,10,"ClearSpec, LLC"
3,Jul 15 2022 1:34PM,241981,20,Alumier Labs Inc.
4,Jul 15 2022 1:20PM,241979,21,"NBTY Global, Inc."
5,Jul 15 2022 1:04PM,241976,19,ACG North America LLC
7,Jul 15 2022 11:21AM,241966,10,ISDIN Corporation
16,Jul 15 2022 10:39AM,241962,12,"Uniderm USA, Inc"
18,Jul 15 2022 10:18AM,241958,10,"Methapharm, Inc."
19,Jul 14 2022 4:35PM,241926,16,Seqirus
20,Jul 14 2022 4:31PM,241925,16,Seqirus


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 15 2022 1:44PM,241983,10,"Citieffe, Inc.",,2
1,Jul 15 2022 1:42PM,241982,10,"ClearSpec, LLC",,1
2,Jul 15 2022 1:34PM,241981,20,Alumier Labs Inc.,,1
3,Jul 15 2022 1:20PM,241979,21,"NBTY Global, Inc.",,1
4,Jul 15 2022 1:04PM,241976,19,ACG North America LLC,,2
5,Jul 15 2022 11:21AM,241966,10,ISDIN Corporation,,9
6,Jul 15 2022 10:39AM,241962,12,"Uniderm USA, Inc",1,1
7,Jul 15 2022 10:18AM,241958,10,"Methapharm, Inc.",,1
8,Jul 14 2022 4:35PM,241926,16,Seqirus,,1
9,Jul 14 2022 4:31PM,241925,16,Seqirus,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 15 2022 1:44PM,241983,10,"Citieffe, Inc.",2,
1,Jul 15 2022 1:42PM,241982,10,"ClearSpec, LLC",1,
2,Jul 15 2022 1:34PM,241981,20,Alumier Labs Inc.,1,
3,Jul 15 2022 1:20PM,241979,21,"NBTY Global, Inc.",1,
4,Jul 15 2022 1:04PM,241976,19,ACG North America LLC,2,
5,Jul 15 2022 11:21AM,241966,10,ISDIN Corporation,9,
6,Jul 15 2022 10:39AM,241962,12,"Uniderm USA, Inc",1,1
7,Jul 15 2022 10:18AM,241958,10,"Methapharm, Inc.",1,
8,Jul 14 2022 4:35PM,241926,16,Seqirus,1,
9,Jul 14 2022 4:31PM,241925,16,Seqirus,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 15 2022 1:44PM,241983,10,"Citieffe, Inc.",2,
1,Jul 15 2022 1:42PM,241982,10,"ClearSpec, LLC",1,
2,Jul 15 2022 1:34PM,241981,20,Alumier Labs Inc.,1,
3,Jul 15 2022 1:20PM,241979,21,"NBTY Global, Inc.",1,
4,Jul 15 2022 1:04PM,241976,19,ACG North America LLC,2,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 15 2022 1:44PM,241983,10,"Citieffe, Inc.",2.0,NaN
1,Jul 15 2022 1:42PM,241982,10,"ClearSpec, LLC",1.0,NaN
2,Jul 15 2022 1:34PM,241981,20,Alumier Labs Inc.,1.0,NaN
3,Jul 15 2022 1:20PM,241979,21,"NBTY Global, Inc.",1.0,NaN
4,Jul 15 2022 1:04PM,241976,19,ACG North America LLC,2.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 15 2022 1:44PM,241983,10,"Citieffe, Inc.",2.0,0.0
1,Jul 15 2022 1:42PM,241982,10,"ClearSpec, LLC",1.0,0.0
2,Jul 15 2022 1:34PM,241981,20,Alumier Labs Inc.,1.0,0.0
3,Jul 15 2022 1:20PM,241979,21,"NBTY Global, Inc.",1.0,0.0
4,Jul 15 2022 1:04PM,241976,19,ACG North America LLC,2.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,967889,13.0,0.0
12,241962,1.0,1.0
16,967694,3.0,1.0
19,241976,2.0,0.0
20,725811,62.0,14.0
21,483885,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,967889,13.0,0.0
1,12,241962,1.0,1.0
2,16,967694,3.0,1.0
3,19,241976,2.0,0.0
4,20,725811,62.0,14.0
5,21,483885,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,13.0,0.0
1,12,1.0,1.0
2,16,3.0,1.0
3,19,2.0,0.0
4,20,62.0,14.0
5,21,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,13.0
1,12,Released,1.0
2,16,Released,3.0
3,19,Released,2.0
4,20,Released,62.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21
Status,,,,,,
Executing,0.0,1.0,1.0,0.0,14.0,0.0
Released,13.0,1.0,3.0,2.0,62.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21
0,Executing,0.0,1.0,1.0,0.0,14.0,0.0
1,Released,13.0,1.0,3.0,2.0,62.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21
0,Executing,0.0,1.0,1.0,0.0,14.0,0.0
1,Released,13.0,1.0,3.0,2.0,62.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()